In [1]:
# Import basic libraries and tools
from functools import lru_cache
from time import time
import numpy as np
from numba import njit
from pprint import pprint
from scipy.special import gamma
import math
import copy

In [2]:
# Import Sympy
from sympy import *
from sympy.functions.special.tensor_functions import KroneckerDelta as KD
from sympy.functions.elementary.trigonometric import sin, cos
from sympy.functions import Abs, sign

In [3]:
# Setup LaTeX printing
# Switches
use_pretty_print = True

# Murray and Dermott G-function'
## use_md_gfunc = True
    
# use NSR
use_nsr = True
show_latex = True

# Set Max l
min_l = 2
max_l = 2

# Set max power of eccentricity
max_e_power = 10

# Performance Notes:
#    Compiling Results for l=2, e=20 on my desktop takes 68952.3862695694 Sec (19 Hours)
#    After ^ that compile, did it for l=3, e=20 ** ** takes 104769.17482447624 Sec (29 Hours) 
#   ^^ for l=7, e=20 ^^ takes 376306.64565110207 (using function, not this notebook) (4.35 days)

max_e_calc_power = max_e_power

max_q = round(max_e_power / 2) + 1

In [4]:
# Script Adjustments
if use_pretty_print:
    init_printing()
    disp = lambda text: display(text)
else:
    disp = print

In [5]:
# Helper Functions
def taylor(func, x, n, x0=0., debug=False, simplify=True):
    if debug:
        return func
    else:
        res = series(func, x, x0, n)
        if simplify:
            res = nsimplify(res)
    return res

In [6]:
@lru_cache(maxsize=2000)
def binomial_coeff(top, bot):
    
    if top == bot:
        return 1
    
    if bot == 0:
        return 1
    
    if bot < 0:
        return 0
    
    if top < 0:
        if bot >= 0:
            return (-1)**bot * binomial_coeff(-top + bot - 1, bot)
        elif bot <= top:
            return (-1)**(top - bot) * binomial_coeff(-bot - 1, top - bot)
        else:
            return 0
    else:
        if 0 <= bot < top:
            return Rational(gamma(top + 1), (gamma(bot + 1) * gamma(top - bot + 1)))
        else:
            return 0
    
    raise Exception()

In [7]:
@lru_cache(maxsize=5000)
def besselj_func(a, x, cutoff):
    
    abs_a = abs(a)
    summation = 0
    m = 0
    while True:
        expo = 2 * m + abs_a
        if expo > cutoff:
            break
            
        this_term = (-1)**m * Rational(1, gamma(m + 1) * gamma(m + abs_a + 1)) * Rational(1, 2**(expo)) * x**(expo)
        summation += this_term
        m += 1
    
    return summation * I**(abs_a - a)

In [8]:
# Symoblic Functions and Variables
ecc = Symbol('e', positive=True, real=True)
inclin_host = Symbol('I___H', real=True)
inclin_sat = Symbol('I___S', real=True)

mean_n = Symbol('n', real=True)
spin_host = Symbol('Omega___H', real=True)
spin_sat = Symbol('Omega___S', real=True)
moi_host = Symbol('C___H', real=True, positive=True)
moi_sat = Symbol('C___S', real=True, positive=True)

love2_num_host = Function('Xi___H2')
love2_num_sat = Function('Xi___S2')
love3_num_host = Function('Xi___H3')
love3_num_sat = Function('Xi___S3')
love4_num_host = Function('Xi___H4')
love4_num_sat = Function('Xi___S4')
love5_num_host = Function('Xi___H5')
love5_num_sat = Function('Xi___S5')
sign_func = Function('Upsilon')

# Planetary and Orbital Variables
mass_host = Symbol('M___H', positive=True, real=True)
mass_sat = Symbol('M___S', positive=True, real=True)
radius_host = Symbol('R___H', positive=True, real=True)
radius_sat = Symbol('R___S', positive=True, real=True)
semi_major_axis = Symbol('a', positive=True, real=True)
newton_G = Symbol('G', positive=True, real=True)

# Make orders
orders = O(ecc**max_e_power)

In [9]:
## Eccentricity Functions
@lru_cache(maxsize=50)
def hansen_wrapper(n, m, k, eccentricity, cutoff_power, force_break: bool = False, going_to_square: bool = True):
    
    # Some hansen numbers can be provided with exact precision (if k==0)
    is_exact = False
    
    # Fix the cutoff power to ensure that precision is maximum while keeping efficiency high.
    #     First we need to use +1 for the taylor since "cutoff" means "last one we WANT to keep"
    cutoff_power_touse = cutoff_power + 1
    if going_to_square and k != 0:
        # An additional efficiency gain can be made by reducing the cutoff threshold if we assume the user is going to 
        #    square the result. Since the minimum e power is equal to q then we never need terms that are > cutoff - q
        q_ = k - m
        cutoff_power_touse -= abs(q_)
    
    if k == 0:
        is_exact = True
         #TODO: is this next line right?
        if m < 0:
            return hansen_wrapper(n, -m, 0, eccentricity, cutoff_power, force_break=force_break, going_to_square=going_to_square)
        
        # When k==0 then the Hansen coefficients are exact (see Laskar and Boue 2010)
        if n == -1:
            # Case where n == -1
            if m == 0:
                return is_exact, 1
            elif m == 1:
                return is_exact, ((1 - eccentricity**2)**(1/2) - 1) / eccentricity
            else:
                return is_exact, 0
        elif n < -1:
            # Case where n < -1
            n_pos = -n
            if m >= n_pos - 1:
                return is_exact, 0
            else:
                summation = 0
                # TODO: is the floor correct here?
                top_sum = math.floor((n_pos - 2 - m) / 2)
                for p in range(0, top_sum+1):
                    if force_break:
                        if (m + 2 * p) > cutoff_power_touse:
                            is_exact = False
                            break
                    coeff = Rational(gamma(n_pos - 2 + 1), gamma(p + 1) * gamma(m + p + 1) * gamma(n_pos - 2 - m - 2*p + 1))
                    eccen = (eccentricity / 2)**(m + 2 * p)
                    summation += coeff * eccen
                return is_exact, ((1 - eccentricity**2)**(-n_pos + Rational(3,2))) * summation
        else:
            # Case where n >= 0
            summation = 0
            # TODO: is the floor correct here?
            top_sum = math.floor((n + 1 - m) / 2)
            for p in range(0, top_sum+1):
                if force_break:
                    if (m + 2 * p) > cutoff_power_touse:
                        is_exact = False
                        break
                coeff = Rational(gamma(1 + n - m + 1), gamma(p + 1) * gamma(m + p + 1) * gamma(1 + n - m - 2*p + 1))
                eccen = (eccentricity / 2)**(m + 2 * p)
                summation += coeff * eccen
            outer_coeff = (-1)**(m) * Rational(gamma(1 + n + m + 1), gamma(1 + n + 1))
            return is_exact, outer_coeff * summation
    else:
        # k != 0 is not exact and requires a truncation on a series (see Renaud et al. 2020)
        if m <= 0 and k < 0:
            return is_exact, hansen_3(n, -m, -k, eccentricity, cutoff_power_touse)
            
        return is_exact, hansen_3(n, m, k, eccentricity, cutoff_power_touse)
  

In [10]:
@lru_cache(maxsize=500)
def hansen_3(a, b, c, eccentricity, cutoff_power):
    
    cutoff_power_touse = cutoff_power
    
    beta = (1 - (1 - eccentricity**2)**(1/2)) / eccentricity
    beta = taylor(beta, eccentricity, cutoff_power_touse).removeO()
        
    outer_sum = 0
    p = 0
    while True:
        if p > cutoff_power_touse + 1:
            break
        
        inner_sum = 0
        for h in range(0, p+1):
            coeff_1 = binomial_coeff(a + b + 1, p - h)
            coeff_2 = binomial_coeff(a - b + 1, h)
            bess = besselj_func(c - b + p - 2 * h, c * eccentricity, cutoff_power_touse)
            inner_sum += coeff_1 * coeff_2 * bess
        
        outer_sum += inner_sum * (-beta)**p
        p += 1
    
    res = (1 + beta**2)**(-a - 1) * outer_sum
    return taylor(res, eccentricity, cutoff_power)

In [11]:
# Eccentricity Functions
@lru_cache(maxsize=50)
def G_func_new(l, p, q, eccentricity, cutoff_power, going_to_square: bool = True):
    return hansen_wrapper(n=-l-1, m=l-2*p, k=l-2*p+q, eccentricity=eccentricity, cutoff_power=cutoff_power, going_to_square=going_to_square)

In [12]:
sin_f_S = Symbol('S___S', real=True)
sin_f_H = Symbol('S___H', real=True)
cos_f_S = Symbol('C___S', real=True)
cos_f_H = Symbol('C___H', real=True)

## Inclination Functions
@lru_cache(maxsize=50)
def F_func(l, m, p, inclination, cut_off_power = None, auto_taylor: bool = False, run_trigsimp: bool = True,
           use_symbol: bool = False):
    lower_sum_bound = max(0, l-m-2*p)
    upper_sum_bound = min(l-m, 2*l - 2*p)
    
    cos_f = None
    sin_f = None
    if use_symbol:
        if inclination is inclin_host:
            sin_f = sin_f_H
            cos_f = cos_f_H
        elif inclination is inclin_sat:
            sin_f = sin_f_S
            cos_f = cos_f_S
    else:
        cos_f = cos(inclination / 2)
        sin_f = sin(inclination / 2)
    
    # See the discussion in Gooding & Wagner 2008 and in the appendix of Renaud+2020. also Veras et al 2019
    outer_coeff = Rational(gamma(l + m + 1), (2**l * gamma(p + 1) * gamma(l - p + 1)))
    
    summation = 0.
    for lam in range(lower_sum_bound, upper_sum_bound+1):
        cos_expo = int(3 * l - m - 2 * p - 2 * lam)
        sin_expo = int(m - l + 2 * p + 2 * lam)
        
        term_1 = (-1)**lam
        term_2 = binomial_coeff(2*l - 2*p, lam)
        term_3 = binomial_coeff(2*p, l - m - lam)
        term_4 = cos_f**cos_expo
        term_5 = sin_f**sin_expo
        
        summation += term_1 * term_2 * term_3 * term_4 * term_5
    
    result = outer_coeff * summation
    if auto_taylor:
        if cut_off_power is None:
            raise Exception('Cutoff Power Required')
        result = taylor(result, inclination, cut_off_power+1).removeO()
    elif run_trigsimp:
        result = simplify(expand_trig(trigsimp(result)))
    
    return result    

In [13]:
print_uni_coeffs = False
if print_uni_coeffs:
    for l in range(7, 7+1):
        for m in range(0, l+1):
            if m == 0:
                universal_coeff = Rational(gamma(1 + l - m), gamma(1 + l + m))
            else:
                universal_coeff = 2*Rational(gamma(1 + l - m), gamma(1 + l + m))
            print(l, m)
            disp(universal_coeff)

In [14]:
print_inclines = False
reduce_fractions = True
tab = '    '

l_ = 2

incline_str = '@njit\ndef calc_inclination(inclination: FloatArray) -> Dict[Tuple[int, int], FloatArray]:\n' + tab + '"""Calculate F^2_lmp for l = ' + f'{l_}' + '"""\n\n' + tab
incline_str += f'# Inclination Functions Calculated for l = {l_}.\n' + tab
incline_str_off = '@njit\ndef calc_inclination_off(inclination: FloatArray) -> Dict[Tuple[int, int], FloatArray]:\n' + tab + '"""Calculate F^2_lmp (assuming I=0) for l = ' + f'{l_}' + '"""\n\n' + tab
incline_str_off += f'# Inclination Functions Calculated for l = {l_}, Inclination == off.\n'

incline_str += f'# Optimizations\n' + tab
incline_str += 'i = inclination\n' + tab
incline_str += 'i_half = i / 2.\n' + tab
incline_str += 'i_double = 2. * i\n' + tab
incline_str += 'i_triple = 3. * i\n' + tab
incline_str += 'sin_i = np.sin(i)\n' + tab
incline_str += 'cos_i = np.cos(i)\n' + tab
incline_str += 'sin_i_half = np.sin(i_half)\n' + tab
incline_str += 'cos_i_half = np.cos(i_half)\n' + tab
incline_str += 'sin_i_double = np.sin(i_double)\n' + tab
incline_str += 'cos_i_double = np.cos(i_double)\n' + tab
incline_str += 'sin_i_triple = np.sin(i_triple)\n' + tab
incline_str += 'cos_i_triple = np.cos(i_triple)\n' + '\n'

incline_str += tab + 'inclination_results = {\n'

incline_str_off += tab + 'ones_ = np.ones_like(inclination)\n\n'
incline_str_off += tab + 'inclination_results = {\n'

for m in range(0, l_ + 1):

    for p in range(0, l_ + 1):


        F2 = trigsimp(F_func(l_, m, p, inclin_sat)**2)
        if print_inclines:
            print(l_, m, p)
            disp(F2)
        F2_off = F2.subs(inclin_sat, 0)
        if reduce_fractions:
            F2 = F2.evalf(25)
            F2_off = F2_off.evalf(25)
        F2_str = str(F2)
        F2_off_str = str(F2_off)
        F2_str = F2_str.replace('sin(I___S/2)', 'sin_i_half')
        F2_str = F2_str.replace('cos(I___S/2)', 'cos_i_half')
        F2_str = F2_str.replace('sin(2*I___S)', 'sin_i_double')
        F2_str = F2_str.replace('cos(2*I___S)', 'cos_i_double')
        F2_str = F2_str.replace('sin(3*I___S)', 'sin_i_triple')
        F2_str = F2_str.replace('cos(3*I___S)', 'cos_i_triple')
        F2_str = F2_str.replace('sin(I___S)', 'sin_i')
        F2_str = F2_str.replace('cos(I___S)', 'cos_i')

        if F2_str == '0':
            pass
        else:
            incline_str += 2*tab + f'({m}, {p}) : ' + F2_str
            if m == l_ and p == l_:
                incline_str += '\n'
            else:
                incline_str += ',\n'

        if F2_off_str == '0':
            pass
        else:
            incline_str_off += 2*tab + f'({m}, {p}) : ' + F2_off_str + ' * ones_'
            if m == l_ and p == l_:
                incline_str_off += '\n'
            else:
                incline_str_off += ',\n'

incline_str += tab + '}\n'
incline_str += '\n' + tab + 'return inclination_results\n'
incline_str_off += tab + '}\n'
incline_str_off += '\n' + tab + 'return inclination_results\n'

file_name = f'NSR Dissipation - TidalPy Output - Inclination Output for {l_}.py'
with open(file_name, 'w') as incline_file:
    incline_file.write(incline_str_off + '\n\n')
    incline_file.write(incline_str)

In [15]:
show_q_check = False
plot_all_qs = False
# Test what q-values we should include.taylor(func, x, n, x0=0., debug=False)

if show_q_check:
    if plot_all_qs:
        q_range = range(-max_q, max_q+1)
    else:
        q_range = [-max_q, max_q]

    for q_ in q_range:
        print(f'\tq = {q_}')
        _, g_p1 = G_func_new(2, 0, q_, ecc, cutoff_power=max_e_calc_power)
        if abs(q_) > 5: 
            print('\tp0 done')
        _, g_p2 = G_func_new(2, 1, q_, ecc, cutoff_power=max_e_calc_power)
        if abs(q_) > 5: 
            print('\tp1 done')
        _, g_p3 = G_func_new(2, 2, q_, ecc, cutoff_power=max_e_calc_power)
        if abs(q_) > 5: 
            print('\tp2 done')
        for g in [g_p1, g_p2, g_p3]:
            print('\tG')
            disp(g)
            print('\tG^2')
            try:
                if g.getO() is None:
                    disp(g**2)
                else:
                    g_ = g.remove()**2
                    disp(taylor(g_, ecc, max_e_power+1))
            except AttributeError:
                disp(g)

In [16]:
## Expression Functions
def sync_spin(expr, host: bool, sat: bool):
    
    if host:
        expr = expr.subs(spin_host, mean_n)
        expr = expr.subs(spin_host, mean_n).subs(sign_func(0), 0).subs(sign_func(-mean_n), -1).subs(sign_func(mean_n), 1)
        expr = expr.subs(love2_num_host(0), 0)
        expr = expr.subs(love3_num_host(0), 0)
        expr = expr.subs(love4_num_host(0), 0)
    
    if sat:
        expr = expr.subs(spin_sat, mean_n)
        expr = expr.subs(spin_sat, mean_n).subs(sign_func(0), 0).subs(sign_func(-mean_n), -1).subs(sign_func(mean_n), 1)
        expr = expr.subs(love2_num_sat(0), 0)
        expr = expr.subs(love3_num_sat(0), 0)
        expr = expr.subs(love4_num_sat(0), 0)
        
    expr = expr.removeO()
    
    for i in range(max_q):
        expr = expr.subs(love2_num_sat(-i*mean_n), -love2_num_sat(i*mean_n))
        expr = expr.subs(love2_num_host(-i*mean_n), -love2_num_host(i*mean_n))
        expr = expr.subs(love2_num_sat(-i*spin_sat), -love2_num_sat(i*spin_sat))
        expr = expr.subs(love2_num_host(-i*spin_host), -love2_num_host(i*spin_host))
        expr = expr.subs(love3_num_sat(-i*mean_n), -love3_num_sat(i*mean_n))
        expr = expr.subs(love3_num_host(-i*mean_n), -love3_num_host(i*mean_n))
        expr = expr.subs(love3_num_sat(-i*spin_sat), -love3_num_sat(i*spin_sat))
        expr = expr.subs(love3_num_host(-i*spin_host), -love3_num_host(i*spin_host))
        expr = expr.subs(love4_num_sat(-i*mean_n), -love4_num_sat(i*mean_n))
        expr = expr.subs(love4_num_host(-i*mean_n), -love4_num_host(i*mean_n))
        expr = expr.subs(love4_num_sat(-i*spin_sat), -love4_num_sat(i*spin_sat))
        expr = expr.subs(love4_num_host(-i*spin_host), -love4_num_host(i*spin_host))
        expr = expr.subs(love5_num_sat(-i*mean_n), -love5_num_sat(i*mean_n))
        expr = expr.subs(love5_num_host(-i*mean_n), -love5_num_host(i*mean_n))
        expr = expr.subs(love5_num_sat(-i*spin_sat), -love5_num_sat(i*spin_sat))
        expr = expr.subs(love5_num_host(-i*spin_host), -love5_num_host(i*spin_host))

        h = float(i)
        expr = expr.subs(love2_num_sat(-h*mean_n), -love2_num_sat(h*mean_n))
        expr = expr.subs(love2_num_host(-h*mean_n), -love2_num_host(h*mean_n))
        expr = expr.subs(love2_num_sat(-h*spin_sat), -love2_num_sat(h*spin_sat))
        expr = expr.subs(love2_num_host(-h*spin_host), -love2_num_host(h*spin_host))
        expr = expr.subs(love3_num_sat(-h*mean_n), -love3_num_sat(h*mean_n))
        expr = expr.subs(love3_num_host(-h*mean_n), -love3_num_host(h*mean_n))
        expr = expr.subs(love3_num_sat(-h*spin_sat), -love3_num_sat(h*spin_sat))
        expr = expr.subs(love3_num_host(-h*spin_host), -love3_num_host(h*spin_host))
        expr = expr.subs(love4_num_sat(-h*mean_n), -love4_num_sat(h*mean_n))
        expr = expr.subs(love4_num_host(-h*mean_n), -love4_num_host(h*mean_n))
        expr = expr.subs(love4_num_sat(-h*spin_sat), -love4_num_sat(h*spin_sat))
        expr = expr.subs(love4_num_host(-h*spin_host), -love4_num_host(h*spin_host))
        expr = expr.subs(love5_num_sat(-h*mean_n), -love5_num_sat(h*mean_n))
        expr = expr.subs(love5_num_host(-h*mean_n), -love5_num_host(h*mean_n))
        expr = expr.subs(love5_num_sat(-h*spin_sat), -love5_num_sat(h*spin_sat))
        expr = expr.subs(love5_num_host(-h*spin_host), -love5_num_host(h*spin_host))
        
    return expr

def make_orbit_boring(expr, reduce_e):
    
    if reduce_e:
        expr = expr.subs(ecc, 0)
    expr = expr.subs(inclin_sat, 0)
    expr = expr.subs(inclin_host, 0)
    return expr.removeO()

def truncate_orbit(expr):
    
    expr += orders
    expr = expr.removeO()
    expr = taylor(expr, inclin_sat, max_i_power+1, debug=False)
    expr = expr.removeO()
    expr = taylor(expr, inclin_host, max_i_power+1, debug=False)
    expr = expr.removeO()
    expr = taylor(expr, ecc, max_e_power+1, debug=False)
    return expr.removeO()

def clean_up(expr, use_factor: bool = True):
    
    for i in range(max_q):
        expr = expr.subs(love2_num_sat(-i*mean_n), -love2_num_sat(i*mean_n))
        expr = expr.subs(love2_num_host(-i*mean_n), -love2_num_host(i*mean_n))
        expr = expr.subs(love2_num_sat(-i*spin_sat), -love2_num_sat(i*spin_sat))
        expr = expr.subs(love2_num_host(-i*spin_host), -love2_num_host(i*spin_host))
        expr = expr.subs(love3_num_sat(-i*mean_n), -love3_num_sat(i*mean_n))
        expr = expr.subs(love3_num_host(-i*mean_n), -love3_num_host(i*mean_n))
        expr = expr.subs(love3_num_sat(-i*spin_sat), -love3_num_sat(i*spin_sat))
        expr = expr.subs(love3_num_host(-i*spin_host), -love3_num_host(i*spin_host))
        expr = expr.subs(love4_num_sat(-i*mean_n), -love4_num_sat(i*mean_n))
        expr = expr.subs(love4_num_host(-i*mean_n), -love4_num_host(i*mean_n))
        expr = expr.subs(love4_num_sat(-i*spin_sat), -love4_num_sat(i*spin_sat))
        expr = expr.subs(love4_num_host(-i*spin_host), -love4_num_host(i*spin_host))
        expr = expr.subs(love5_num_sat(-i*mean_n), -love5_num_sat(i*mean_n))
        expr = expr.subs(love5_num_host(-i*mean_n), -love5_num_host(i*mean_n))
        expr = expr.subs(love5_num_sat(-i*spin_sat), -love5_num_sat(i*spin_sat))
        expr = expr.subs(love5_num_host(-i*spin_host), -love5_num_host(i*spin_host))

        h = float(i)
        expr = expr.subs(love2_num_sat(-h*mean_n), -love2_num_sat(h*mean_n))
        expr = expr.subs(love2_num_host(-h*mean_n), -love2_num_host(h*mean_n))
        expr = expr.subs(love2_num_sat(-h*spin_sat), -love2_num_sat(h*spin_sat))
        expr = expr.subs(love2_num_host(-h*spin_host), -love2_num_host(h*spin_host))
        expr = expr.subs(love3_num_sat(-h*mean_n), -love3_num_sat(h*mean_n))
        expr = expr.subs(love3_num_host(-h*mean_n), -love3_num_host(h*mean_n))
        expr = expr.subs(love3_num_sat(-h*spin_sat), -love3_num_sat(h*spin_sat))
        expr = expr.subs(love3_num_host(-h*spin_host), -love3_num_host(h*spin_host))
        expr = expr.subs(love4_num_sat(-h*mean_n), -love4_num_sat(h*mean_n))
        expr = expr.subs(love4_num_host(-h*mean_n), -love4_num_host(h*mean_n))
        expr = expr.subs(love4_num_sat(-h*spin_sat), -love4_num_sat(h*spin_sat))
        expr = expr.subs(love4_num_host(-h*spin_host), -love4_num_host(h*spin_host))
        expr = expr.subs(love5_num_sat(-h*mean_n), -love5_num_sat(h*mean_n))
        expr = expr.subs(love5_num_host(-h*mean_n), -love5_num_host(h*mean_n))
        expr = expr.subs(love5_num_sat(-h*spin_sat), -love5_num_sat(h*spin_sat))
        expr = expr.subs(love5_num_host(-h*spin_host), -love5_num_host(h*spin_host))
    expr = expr.subs(love2_num_host(0), 0)
    expr = expr.subs(love3_num_host(0), 0)
    expr = expr.subs(love4_num_host(0), 0)
    expr = expr.subs(love5_num_host(0), 0)
    expr = expr.subs(love2_num_sat(0), 0)
    expr = expr.subs(love3_num_sat(0), 0)
    expr = expr.subs(love4_num_sat(0), 0)
    expr = expr.subs(love5_num_sat(0), 0)
    expr = expr.removeO()
    return expr

def disp_wclean(expr):
    
    disp(clean_up(expr))
    
def flip_a2n(expr):
    return expr.subs(semi_major_axis, (newton_G*(mass_host + mass_sat)/mean_n**2)**(1/3))
    
def remove_G(expr):
    return expr.subs(newton_G, semi_major_axis**3*mean_n**2/(mass_host + mass_sat))

def collect_ei(expr):
    
    expr = collect(expr, ecc)
    expr = collect(expr, inclin_host)
    expr = collect(expr, inclin_sat)
    
    return expr

def collect_mode(expr):
    
    for lmpq, nimk in omega_table_host.items():
        expr = collect(expr, nimk)
    
    for lmpq, nimk in omega_table_sat.items():
        expr = collect(expr, nimk)
    
#     for _, mode in omega_table_host.items():
#         expr = collect(expr, love_num_host(mode))
    
#     for _, mode in omega_table_sat.items():
#         expr = collect(expr, love_num_sat(mode))
    
    return expr

In [17]:
print_list = False
build_tables = True

g_table = dict()
f_table_host = dict()
f_table_sat = dict()
g2_table = dict()
f2_table_host = dict()
f2_table_sat = dict()
omega_table_host = dict()
omega_table_sat = dict()
freq_table_host = dict()
freq_table_sat = dict()
sign_table_host = dict()
sign_table_sat = dict()
love_table_host = dict()
love_table_sat = dict()
love_omg_table_host = dict()
love_omg_table_sat = dict()

init_time = time()

if print_list or build_tables:
    for l in range(min_l, max_l+1):
        if l==2:
            love_func_host = love2_num_host
            love_func_sat = love2_num_sat
        elif l==3:
            love_func_host = love3_num_host
            love_func_sat = love3_num_sat
        elif l==4:
            love_func_host = love4_num_host
            love_func_sat = love4_num_sat
        elif l==5:
            love_func_host = love5_num_host
            love_func_sat = love5_num_sat
        else:
            raise NotImplemented
    
        for m in range(0, l+1):
            for p in range(0, l+1):
                if build_tables:
                    f_table_host[(l, m, p)] = F_func(l, m, p, inclin_host, auto_taylor=False,
                                                     run_trigsimp=False, use_symbol=False)
                    f_table_sat[(l, m, p)] = F_func(l, m, p, inclin_sat, auto_taylor=False,
                                                    run_trigsimp=False, use_symbol=False)
#                     f2_table_host[(l, m, p)] = taylor(f_table_host[(l, m, p)] * f_table_host[(l, m, p)], inclin_host, max_i_power+1)
#                     f2_table_sat[(l, m, p)] = taylor(f_table_sat[(l, m, p)] * f_table_sat[(l, m, p)], inclin_sat, max_i_power+1)
                    f2_table_host[(l, m, p)] = simplify(expand(f_table_host[(l, m, p)] * f_table_host[(l, m, p)]))
                    f2_table_sat[(l, m, p)] = simplify(expand(f_table_sat[(l, m, p)] * f_table_sat[(l, m, p)]))
#                     f2_table_host[(l, m, p)] = trigsimp(f2_table_host[(l, m, p)])
#                     f2_table_sat[(l, m, p)] = trigsimp(f2_table_sat[(l, m, p)])
        
                for q in range(-max_q, max_q+1):
                    
                    if m == 0:
                        q_init_time = time()
                        print(f'Building: ({l}, {m}, {p}, {q})  ')
                        
                        is_exact, g_table[(l, p, q)] = G_func_new(l, p, q, ecc, max_e_calc_power)
                        
                        if is_exact:
                            g2_table[(l, p, q)] = simplify(g_table[(l, p, q)]**2)
                        else:
                            g2_table[(l, p, q)] = taylor(g_table[(l, p, q)].removeO()**2, ecc, max_e_power+1)
                        
                        print('\tMode Time: ', time() - q_init_time)

                    omega_table_host[(l, m, p, q)] = (l - 2*p + q)*mean_n - m*spin_host
                    omega_table_sat[(l, m, p, q)] = (l - 2*p + q)*mean_n - m*spin_sat
                    freq_table_host[(l, m, p, q)] = Abs(omega_table_host[(l, m, p, q)])
                    freq_table_sat[(l, m, p, q)] = Abs(omega_table_sat[(l, m, p, q)])
                    sign_table_host[(l, m, p, q)] = sign(omega_table_host[(l, m, p, q)])
                    sign_table_sat[(l, m, p, q)] = sign(omega_table_sat[(l, m, p, q)])

                    if omega_table_host[(l, m, p, q)] == 0.:
                        love_table_host[(l, m, p, q)] = 0.
                        love_omg_table_host[(l, m, p, q)] = 0.
                    else:
                        love_table_host[(l, m, p, q)] = love_func_host(freq_table_host[(l, m, p, q)])
                        love_omg_table_host[(l, m, p, q)] = love_func_host(omega_table_host[(l, m, p, q)])
                    if omega_table_sat[(l, m, p, q)] == 0.:
                        love_table_sat[(l, m, p, q)] = 0.
                        love_omg_table_sat[(l, m, p, q)] = 0.
                    else:
                        love_table_sat[(l, m, p, q)] = love_func_sat(freq_table_sat[(l, m, p, q)])
                        love_omg_table_sat[(l, m, p, q)] = love_func_sat(omega_table_sat[(l, m, p, q)])
                    
                    if print_list:
                        disp('')
                        print(f'\nl, m, p, q = {l}{m}{p}{q}')
                        print('\tMode_h=')
                        disp(nsimplify(omega_table_host[(l, m, p, q)]))
                        print('\tG(e)=')
                        disp(nsimplify(g_table[(l, p, q)]))
                        print('\tF_h(I)=')
                        disp(nsimplify(f_table_host[(l, m, p)]))

print('Total Time: ', time() - init_time)

Building: (2, 0, 0, -6)  
	Mode Time:  0.3152954578399658
Building: (2, 0, 0, -5)  
	Mode Time:  0.2812469005584717
Building: (2, 0, 0, -4)  
	Mode Time:  0.23621606826782227
Building: (2, 0, 0, -3)  
	Mode Time:  0.6686077117919922
Building: (2, 0, 0, -2)  
	Mode Time:  0.0010004043579101562
Building: (2, 0, 0, -1)  
	Mode Time:  1.7116162776947021
Building: (2, 0, 0, 0)  
	Mode Time:  1.8470633029937744
Building: (2, 0, 0, 1)  
	Mode Time:  1.218109130859375
Building: (2, 0, 0, 2)  
	Mode Time:  1.2681548595428467
Building: (2, 0, 0, 3)  
	Mode Time:  0.8327763080596924
Building: (2, 0, 0, 4)  
	Mode Time:  0.5404918193817139
Building: (2, 0, 0, 5)  
	Mode Time:  0.3433046340942383
Building: (2, 0, 0, 6)  
	Mode Time:  0.2452232837677002
Building: (2, 0, 1, -6)  
	Mode Time:  0.1621396541595459
Building: (2, 0, 1, -5)  
	Mode Time:  0.40439701080322266
Building: (2, 0, 1, -4)  
	Mode Time:  0.421891450881958
Building: (2, 0, 1, -3)  
	Mode Time:  0.7827095985412598
Building: (2, 0, 1

In [18]:
print(binomial_coeff.cache_info())
print(besselj_func.cache_info())
print(hansen_wrapper.cache_info())
print(hansen_3.cache_info())
print(G_func_new.cache_info())

CacheInfo(hits=2256, misses=65, maxsize=2000, currsize=65)
CacheInfo(hits=742, misses=371, maxsize=5000, currsize=371)
CacheInfo(hits=0, misses=40, maxsize=50, currsize=40)
CacheInfo(hits=14, misses=22, maxsize=500, currsize=22)
CacheInfo(hits=0, misses=39, maxsize=50, currsize=39)


## Tidal Potential Derivative Calculations

In [19]:
physical_coeffs_sat = dict()
physical_coeffs_host = dict()
eccen_inclin_coeffs_sat = dict()
love_coeffs_sat = dict()
eccen_inclin_coeffs_host = dict()
love_coeffs_host = dict()
universal_coeffs = dict()
dUdM_coeffs = dict()
dUdw_coeffs = dict()
dUdO_coeffs = dict()
full_potential_coeffs = dict()

for l in range(min_l, max_l+1):
    for m in range(0, l+1):
        for p in range(0, l+1):            
            F_H = f2_table_host[(l, m, p)]
            F_S = f2_table_sat[(l, m, p)]
            
            for q in range(-max_q, max_q+1):
                
                sgn_H = sign_table_host[(l, m, p, q)]
                sgn_S = sign_table_sat[(l, m, p, q)]
                freq_H = freq_table_host[(l, m, p, q)]
                freq_S = freq_table_sat[(l, m, p, q)]
                omg_H = omega_table_host[(l, m, p, q)]
                omg_S = omega_table_sat[(l, m, p, q)]
                love_H = love_table_host[(l, m, p, q)]
                love_S = love_table_sat[(l, m, p, q)]
                love_omg_H = love_omg_table_host[(l, m, p, q)]
                love_omg_S = love_omg_table_sat[(l, m, p, q)]
                
                G_ = g2_table[(l, p, q)]
                
                eccen_inclin_coeffs_host[(l, m, p, q)] = factor(expand(G_ * F_H).removeO())
                eccen_inclin_coeffs_sat[(l, m, p, q)] = factor(expand(G_ * F_S).removeO())
                
                love_coeffs_sat[(l, m, p, q)] = love_S * sgn_S
                love_coeffs_host[(l, m, p, q)] = love_H * sgn_H
                
                dUdM_coeffs[(l, m, p, q)] = l - 2*p + q
                dUdw_coeffs[(l, m, p, q)] = l - 2*p
                dUdO_coeffs[(l, m, p, q)] = m
                full_potential_coeffs[(l, m, p, q)] = 1
                
                if m == 0:
                    universal_coeffs[(l, m, p, q)] = Rational(gamma(1 + l - m), gamma(1 + l + m))
                else:
                    universal_coeffs[(l, m, p, q)] = 2*Rational(gamma(1 + l - m), gamma(1 + l + m))
                
                # For the purposes of putting coefficients into TidalPy, I want every thing reduced by a factor of 
                #    (3 / 2) as that will be multipled by tidal susceptibility in a later step. 
                universal_coeffs[(l, m, p, q)] /= Rational(3, 2)
                
                physical_coeffs_host[(l, m, p, q)] = \
                    ((radius_host / semi_major_axis)**(2*l + 1) * newton_G * mass_sat) / semi_major_axis
                physical_coeffs_sat[(l, m, p, q)] = \
                    ((radius_sat / semi_major_axis)**(2*l + 1) * newton_G * mass_host) / semi_major_axis
                
print('Completed Coeff Tables Calculations')

Completed Coeff Tables Calculations


## Print and save Eccentricity Results

In [20]:
save_tidalpy_data = True
max_e_toconvert_power_sign = 6
reduce_to_floats = True
if save_tidalpy_data:
    
    import os
    
    output_file = f'NSR Dissipation - TidalPy Output - min_order_l {min_l} - max_e {max_e_power}.py'
    
    tab = '    '
    eccentricity_section = '# Unique results.\neccentricity_results_bymode = {\n'
    eccentricity_dupes_section = '\n# Duplicate results are stored as dictionary lookups to previous calculations\n#    Generally leads to a 30--50% speed-up when working with large arrays.\n'
    eccentricity_preamble = f'# Eccentricity functions calculated at truncation level {max_e_power}.\n' +\
                            f'#     and order-l = {min_l}.\n\n'
    needed_eccens = dict()
    eccens_grabbed = list()
    unique_funcs = dict()
    max_e_text_len = 1
    for i in range(2, max_e_power+1, 2):
        needed_eccens[f'e**{i}'] = f'e{i}'
        max_e_text_len = max(max_e_text_len, len(f'e{i}'))
    
    num_eccen = len(g2_table)
    curr_eccen = 0
    
    for l in range(min_l, min_l+1):
        if max_l != min_l:
            raise Exception('TidalPy Printing only designed to work on one "l" at a time.')
            
        for p in range(0, l+1):

            # Make a new p subsection
            eccentricity_section += tab + f'{p}: ' + '{\n'
            
            # Need to include at least one item in each p otherwise njit will not compile correctly
            p_used = False

            for q in range(-max_q, max_q+1):

                gfunc = g2_table[(l, p, q)].removeO()
                if (l, p, q) not in eccens_grabbed:
                    eccens_grabbed.append((l, p, q))

                    # Clean up the equation text
                    if reduce_to_floats:
                        gfunc_clean = str(gfunc.evalf())
                    else:
                        gfunc_clean = str(gfunc)
                    for old_text in list(reversed(sorted(needed_eccens.keys()))):
                            # Get rid of "e**N" in favor of "eN" which are precomputed
                            new_text = needed_eccens[old_text]
                            gfunc_clean = gfunc_clean.replace(old_text, new_text)

                    if gfunc_clean not in unique_funcs or not p_used:
                        
                        if gfunc_clean == '0':
                            # Ignore 0's
                            continue

                        # Unique equation
                        unique_funcs[gfunc_clean] = (l, p, q)


                        # Record the new equation
                        if q < 0:
                            mode_text = f'{q}: '
                        else:
                            mode_text = f'{q}:  '
                        eccentricity_section += 2*tab + mode_text + gfunc_clean
                        if curr_eccen == num_eccen - 1:
                            eccentricity_section += '\n}\n'
                        else:
                            eccentricity_section += ',\n'
                            
                        # Note that at least one thing was added to this p-dict
                        p_used = True
                    else:
                        
                        # If it is a duplicate, simply point to the old equation to save on computation
                        if gfunc_clean == '0':
                            # Ignore 0's
                            continue
                        
                        old_copy_key = unique_funcs[gfunc_clean]
                        old_l, old_p, old_q = old_copy_key
                        eccentricity_dupes_section += f'eccentricity_results_bymode[{p}][{q}] = eccentricity_results_bymode[{old_p}][{old_q}]\n'

                    curr_eccen += 1

            # End the previous p section
            if p == l:
                eccentricity_section += tab + '}\n}'
            else:
                eccentricity_section += tab + '},\n'
                    
                    
    eccentricity_preamble += '# Performance and readability improvements\n'
    eccentricity_preamble += 'e = eccentricity\n'    
    for old_text, new_text in needed_eccens.items():
        for max_e_conversion in range(2, max_e_toconvert_power_sign+2, 2):
            if f'**{max_e_conversion}' == old_text[-3:]:
                old_text = ' * '.join(list('e'*max_e_conversion))
                break
        eccentricity_preamble += f'{new_text}' + ' ' * (max_e_text_len-len(new_text)) + f' = {old_text}\n'
        
    eccentricity_section = eccentricity_preamble + '\n' + eccentricity_section + '\n' + eccentricity_dupes_section
    
    with open(output_file, 'w') as output_file:
        output_file.write(eccentricity_section)

In [21]:
save_latex_table = True
if save_latex_table:
    
    unique_modes = list()
    unique_freqs = list()
    max_eq_length = 100
    
    table_preamble = \
        '\\begin{landscape}\n' + \
        '\\fudge{3cm}{1.5cm} % 3cm longer and raise by 1.5cm\n' + \
        '\\begin{longtable}[c]{@{}c|c|cccc|c|l@{}}\n' + \
        '    \\caption{Caption \\label{tab:dissipation_order10}} \\\\ \\toprule\n' + \
        '    \tMode & Signature & \\multicolumn{4}{c|}{Coefficients, $C_{X}$} & Inclination Function & Eccentricity Function \\\\\n' + \
        '    $\\omega_{j}$ & $l$, $m$, $p$, $q$ & $\\frac{dU_{j}}{d\\mathcal{M}}$ & $\\frac{dU_{j}}{d\\varpi_{j}}$ & $\\frac{dU_{j}}{d\\Omega_{j}}$ & $\\dot{E}_{j}$ & $F^{2}$ & $G^{2}$ \\\\ \\midrule\n' + \
        '    \\midrule           % line head body\n' + \
        '    \\endfirsthead      % Definition of 1. table header\n' + \
        '    \\toprule\n' + \
        '    \\multicolumn{8}{c}{continue table}\\\\\n' + \
        '    Mode & Signature & \\multicolumn{4}{c|}{Coefficients, $C_{X}$} & Inclination Function & Eccentricity Function \\\\\n' + \
        '    $\\omega_{j}$ & $l$, $m$, $p$, $q$ & $\\frac{dU_{j}}{d\\mathcal{M}}$ & $\\frac{dU_{j}}{d\\varpi_{j}}$ & $\\frac{dU_{j}}{d\\Omega_{j}}$ & $\\dot{E}_{j}$ & $F^{2}$ & $G^{2}$ \\\\ \\midrule\n' + \
        '    \\midrule           % line head body\n' + \
        '    \\endhead      % Definition of all following headers\n' + \
        '    \\midrule\n' + \
        '    \\multicolumn{8}{c}{table continues} \\\\ % footer 1. (and more) part(s) of table\n' + \
        '    \\midrule\n' + \
        '    \\endfoot      % foots of the table without the last one\n' + \
        '    \\bottomrule\n' + \
        '    \\endlastfoot  % the last(!!) foot of the table  %%%%%\n' 
    
    table_postamble = \
        '\\end{longtable}\n' + \
        '\\end{landscape}\n'
    
    
    mode_str = ''
    
    for l in range(min_l, max_l+1):
        for m in range(0, l+1):
            for p in range(0, l+1):
                
                this_incline_grabbed = False
                ffunc = f2_table_sat[(l, m, p)]
                ffunc = simplify(ffunc)
                
                for q in range(-max_q, max_q+1):
                    
                    gfunc = g2_table[(l, p, q)].removeO()
                    
                    # Check if G or F funcs are zero. skip if they are
                    if ffunc == 0 or gfunc == 0:
                        continue
                    
                    # If not skipped then we need to add a line. Add the tab
                    mode_str += '\n'  # Line break before new mode.
                    mode_str += f'    % Mode (lmpq) = ({l}, {m}, {p}, {q})\n' # Add a comment for easy searching within the latex
                    mode_str += '    '
                    
                    # Then the tidal mode
                    omg_S = omega_table_sat[(l, m, p, q)]
                    
                    if omg_S == 0:
                        continue
                        
                    omg_S_str = str(omg_S).replace('Omega___S', '\\ddot{\\theta}_{j}')
                    omg_S_str = omg_S_str.replace('*', '')
                    mode_str += '$' + omg_S_str + '$'
                    
                    # Store unique modes and freqs
                    if omg_S not in unique_modes:
                        unique_modes.append(omg_S)
                    freq_S = expand(abs(omg_S))
                    if freq_S not in unique_freqs:
                        unique_freqs.append(freq_S)
                    
                    # Next the lmpq
                    if q < 0:
                        mode_str += ' & ' + f'{l}, {m}, {p}, {q}'
                    else:
                        mode_str += ' & ' + f'{l}, {m}, {p}, \;{q}'
                    
                    # Then the coefficients
                    dUdM_coeff = dUdM_coeffs[(l, m, p, q)]
                    dUdw_coeff = dUdw_coeffs[(l, m, p, q)]
                    dUdO_coeff = dUdO_coeffs[(l, m, p, q)]
                    heating_coeff = 1
                    uni_coeff = universal_coeffs[(l, m, p, q)]
                    dUdM_coeff *= uni_coeff
                    dUdw_coeff *= uni_coeff
                    dUdO_coeff *= uni_coeff
                    heating_coeff *= uni_coeff
                    
                    mode_str += ' & $' + latex(dUdM_coeff) + '$ & $' + latex(dUdw_coeff) + '$ & $' + latex(dUdO_coeff) + \
                        '$ & $' + latex(heating_coeff) + '$ & \n'
                    mode_str += '    '
                    
                    # Now the inclination functions
                    if not this_incline_grabbed:
                        
                        # Clean up inclination function string
                        incline_str = latex(ffunc).replace('_{S}', '_{j}')
                        incline_str = incline_str.replace('I^{}', 'I')
                        incline_str = incline_str.replace('\\frac{I_{j}}{2}', 'I_{j}/2')
                        
                        
                        mode_str += '\\begin{math}\n    \\begin{aligned}\n        & '
                        # Check length?
                        mode_str += incline_str
                        mode_str += ' \n    \\end{aligned}\n    \end{math} & \n'
                        this_incline_grabbed = True
                    else:
                        mode_str += '--- & \n'
                    
                    # Finally the eccentricity functions
                    mode_str += '    '
                    eccen_str = latex(gfunc)
                    eccen_str = eccen_str.replace('e^{}', 'e')
                    
                    # Check if the eccentricity function needs to be split onto multiple lines.
                    eccen_lines = 1
                    if len(eccen_str) > max_eq_length:
                        
                        term_list = list()
                        # First split along positives
                        for term in eccen_str.split(' + '):
                            if ' - ' in term:
                                # And then along negatives
                                for t_i, neg_term in enumerate(term.split(' - ')):
                                    if t_i == 0:
                                        # First term is the positive term
                                        term_list.append((neg_term, True))
                                    else:
                                        term_list.append((neg_term, False))
                            else:
                                term_list.append((term, True))
                        
                        # Make new string
                        new_eccen_str = ''
                        running_num = 0
                        for t_i, (term, leading_pos) in enumerate(term_list):
                            if t_i > 0:
                                if leading_pos:
                                    term = ' + ' + term.strip()
                                else:
                                    term = ' - ' + term.strip()
                            else:
                                term = term.strip()
                                    
                            running_num += len(term)
                            if running_num > max_eq_length:
                                new_eccen_str += ' \\\\ \n        &' + term
                                running_num = len(term)
                                eccen_lines += 1
                            else:
                                new_eccen_str += term
                                
                        eccen_str = new_eccen_str
                    
                    # The minipage enviornment allows there to be padding above and below the multi line equations (only thing I could get to work)
                    #    The 80mm probably needs to be adjusted if you give the equations more/less horizontal room.
                    mode_str += '\\begin{minipage}{80mm}\n    \\vspace{2mm}    \\begin{math}\n    \\begin{aligned}\n        & '
                    # Check length?
                    mode_str += eccen_str
                    mode_str += ' \n    \\end{aligned}\n    \\end{math}\n    \\vspace{2mm}\n    \\end{minipage} \\\\ \\hline\n'
                    
    # Save as text file
    with open(f'NSR Dissipation - Latex Output - emax_{max_e_power}_minl_{min_l}.txt', 'w') as latex_file:
        output_str = table_preamble + mode_str + table_postamble
        latex_file.write(output_str)

    print('Done!')
    print('# Unique Modes:', len(unique_modes))
    print('# Unique Frequencies:', len(unique_freqs))

Done!
# Unique Modes: 44
# Unique Frequencies: 37


In [22]:
breakpoint()

--Call--
> c:\programdata\anaconda3\lib\site-packages\ipython\core\displayhook.py(252)__call__()
-> def __call__(self, result=None):
(Pdb) q


BdbQuit: 

### Compile Derivatives

In [23]:
beta = (mass_host * mass_sat) / (mass_host + mass_sat)

# Host
torque_H = 0.
heating_H = 0.
dUdM_H = 0.
dUdw_H = 0.
dUdO_H = 0.
for mode, love_coeff in love_coeffs_host.items():
    ei_coeff = eccen_inclin_coeffs_host[mode]
    uni_coeff = universal_coeffs[mode]
    phys_coeff = physical_coeffs_host[mode]
    dUdM_coeff = dUdM_coeffs[mode]
    dUdw_coeff = dUdw_coeffs[mode]
    dUdO_coeff = dUdO_coeffs[mode]
    freq = freq_table_host[mode]
    sgn_H = sign_table_host[mode]
    love_H = love_table_host[mode]
    
    heating_H += expand(Rational(3, 2) * ei_coeff * uni_coeff * phys_coeff * freq * love_H * mass_sat)
    torque_H += expand(Rational(3, 2) * dUdO_coeff * ei_coeff * uni_coeff * phys_coeff * love_coeff * mass_sat)
    dUdM_H += expand(Rational(3, 2) * dUdM_coeff * ei_coeff * uni_coeff * phys_coeff * love_coeff)
    dUdw_H += expand(Rational(3, 2) * dUdw_coeff * ei_coeff * uni_coeff * phys_coeff * love_coeff)
    dUdO_H += expand(Rational(3, 2) * dUdO_coeff * ei_coeff * uni_coeff * phys_coeff * love_coeff)
    
    
# Satellite
heating_S = 0
torque_S = 0
dUdM_S = 0
dUdw_S = 0
dUdO_S = 0
full_potential = 0
heating_S_simp = 0
torque_S_simp = 0
dUdM_S_simp = 0
dUdw_S_simp = 0
dUdO_S_simp = 0
full_potential_simp = 0
for mode, love_coeff in love_coeffs_sat.items():
    ei_coeff = eccen_inclin_coeffs_sat[mode]
    uni_coeff = universal_coeffs[mode]
    phys_coeff = physical_coeffs_sat[mode]
    dUdM_coeff = dUdM_coeffs[mode]
    dUdw_coeff = dUdw_coeffs[mode]
    dUdO_coeff = dUdO_coeffs[mode]
    fullpot_coeff = full_potential_coeffs[mode]
    freq = freq_table_sat[mode]
    sgn_S = sign_table_sat[mode]
    love_S = love_table_sat[mode]
    
    heating_S += expand(Rational(3, 2) * ei_coeff * uni_coeff * phys_coeff * freq * love_S * mass_host)
    torque_S += expand(Rational(3, 2) * dUdO_coeff * ei_coeff * uni_coeff * phys_coeff * love_coeff * mass_host)
    dUdM_S += expand(Rational(3, 2) * dUdM_coeff * ei_coeff * uni_coeff * phys_coeff * love_coeff)
    dUdw_S += expand(Rational(3, 2) * dUdw_coeff * ei_coeff * uni_coeff * phys_coeff * love_coeff)
    dUdO_S += expand(Rational(3, 2) * dUdO_coeff * ei_coeff * uni_coeff * phys_coeff * love_coeff)
    full_potential +=  expand(Rational(3, 2) * fullpot_coeff * ei_coeff * uni_coeff * phys_coeff * love_coeff)
    
    heating_S_simp += expand(ei_coeff * uni_coeff * freq * love_S)
    torque_S_simp += expand(dUdO_coeff * ei_coeff * uni_coeff * love_coeff)
    dUdM_S_simp += expand(dUdM_coeff * ei_coeff * uni_coeff * love_coeff)
    dUdw_S_simp += expand(dUdw_coeff * ei_coeff * uni_coeff * love_coeff)
    dUdO_S_simp += expand(dUdO_coeff * ei_coeff * uni_coeff * love_coeff)
    full_potential_simp += expand(fullpot_coeff * ei_coeff * uni_coeff * love_coeff)
    
    
# Check if the user wanted the objects to both be forced into spin-sync from the get go.
if not use_nsr:
    heating_H = sync_spin(heating_H, host=True, sat=False)
    torque_H = sync_spin(torque_H, host=True, sat=False)
    dUdM_H = sync_spin(dUdM_H, host=True, sat=False)
    dUdw_H = sync_spin(dUdw_H, host=True, sat=False)
    dUdO_H = sync_spin(dUdO_H, host=True, sat=False)
    
    heating_S = sync_spin(heating_S, host=False, sat=True)
    torque_S = sync_spin(torque_S, host=False, sat=True)
    dUdM_S = sync_spin(dUdM_S, host=False, sat=True)
    dUdw_S = sync_spin(dUdw_S, host=False, sat=True)
    dUdO_S = sync_spin(dUdO_S, host=False, sat=True)
    

## Inclination = 0

In [ ]:
# print('Full Potential')
# expr = full_potential_simp
# # expr = taylor(expr, inclin_sat, 3)
# expr = expr.subs(inclin_sat, 0)
# expr = expand(sync_spin(expr, host=True, sat=True))
# for mode, love_S in love_table_sat.items():
#     expr = collect(expr, love_S)
# disp(expr)
# latex_str = latex(expr)
# latex_str = latex_str.replace('\\Xi^{}_{S2}', '\\operatorname{K}_{j}')
# print(latex_str)

In [28]:
print('Tidal Heating')
expr=heating_S_simp
expr = expr.subs(inclin_sat, 0)
expr = expand(sync_spin(expr, host=True, sat=True) / abs(mean_n))
# for mode, love_S in love_table_sat.items():
#     expr = collect(expr, love_S)
disp(expr)
latex_str = latex(expr)
latex_str = latex_str.replace('\\Xi^{}_{S2}', '\\operatorname{K}_{j}')
print(latex_str)

Tidal Heating


\frac{2555911 e^{10} \operatorname{K}_{j}{\left(\left|{n}\right| \right)}}{122880} - \frac{171083 e^{10} \operatorname{K}_{j}{\left(2 \left|{n}\right| \right)}}{320} + \frac{368520907 e^{10} \operatorname{K}_{j}{\left(3 \left|{n}\right| \right)}}{81920} - \frac{66268493 e^{10} \operatorname{K}_{j}{\left(4 \left|{n}\right| \right)}}{5760} + \frac{6576742601 e^{10} \operatorname{K}_{j}{\left(5 \left|{n}\right| \right)}}{737280} - \frac{63949 e^{8} \operatorname{K}_{j}{\left(\left|{n}\right| \right)}}{2304} + \frac{339187 e^{8} \operatorname{K}_{j}{\left(2 \left|{n}\right| \right)}}{576} - \frac{1709915 e^{8} \operatorname{K}_{j}{\left(3 \left|{n}\right| \right)}}{768} + \frac{2592379 e^{8} \operatorname{K}_{j}{\left(4 \left|{n}\right| \right)}}{1152} + \frac{551 e^{6} \operatorname{K}_{j}{\left(\left|{n}\right| \right)}}{12} - \frac{3847 e^{6} \operatorname{K}_{j}{\left(2 \left|{n}\right| \right)}}{12} + \frac{2855 e^{6} \operatorname{K}_{j}{\left(3 \left|{n}\right| \right)}}{6} - \frac{

In [ ]:
# print('dU_dM')
# expr = dUdM_S_simp.subs(inclin_sat, 0)
# expr = expand(sync_spin(expr, host=True, sat=True) / (sign(mean_n)))
# for mode, love_S in love_table_sat.items():
#     expr = collect(expr, love_S)
# disp(expr)
# latex_str = latex(expr)
# latex_str = latex_str.replace('\\Xi^{}_{S2}', '\\operatorname{K}_{j}')
# print(latex_str)

In [ ]:
# print('dU_dw')
# expr = dUdw_S_simp.subs(inclin_sat, 0)
# expr = expand(sync_spin(expr, host=True, sat=True) / (sign(mean_n)))
# for mode, love_S in love_table_sat.items():
#     expr = collect(expr, love_S)
# disp(expr)
# latex_str = latex(expr)
# latex_str = latex_str.replace('\\Xi^{}_{S2}', '\\operatorname{K}_{j}')
# print(latex_str)

In [ ]:
# print('dU_dO')
# expr = dUdO_S_simp.subs(inclin_sat, 0)
# expr = expand(sync_spin(expr, host=True, sat=True) / (sign(mean_n)))
# for mode, love_S in love_table_sat.items():
#     expr = collect(expr, love_S)
# disp(expr)
# latex_str = latex(expr)
# latex_str = latex_str.replace('\\Xi^{}_{S2}', '\\operatorname{K}_{j}')
# print(latex_str)

## Eccentricity = 0

In [ ]:
# print('Full Potential')
# expr = full_potential_simp
# expr = expr.subs(ecc, 0)
# expr = expand(sync_spin(expr, host=True, sat=True))
# for mode, love_S in love_table_sat.items():
#     expr = collect(expr, love_S)
# disp(expr)
# latex_str = latex(expr)
# latex_str = latex_str.replace('\\Xi^{}_{S2}', '\\operatorname{K}_{j}')
# print(latex_str)

In [ ]:
# print('Tidal Heating')
# expr = heating_S_simp
# expr = expr.subs(ecc, 0)
# expr = expand(sync_spin(expr, host=True, sat=True) / abs(mean_n))
# for mode, love_S in love_table_sat.items():
#     expr = collect(expr, love_S)
# disp(expr)
# latex_str = latex(expr)
# latex_str = latex_str.replace('\\Xi^{}_{S2}', '\\operatorname{K}_{j}')
# print(latex_str)

In [ ]:
# print('dU_dM')
# expr = dUdM_S_simp.subs(ecc, 0)
# expr = expand(sync_spin(expr, host=True, sat=True) / (sign(mean_n)))
# for mode, love_S in love_table_sat.items():
#     expr = collect(expr, love_S)
# disp(expr)
# latex_str = latex(expr)
# latex_str = latex_str.replace('\\Xi^{}_{S2}', '\\operatorname{K}_{j}')
# print(latex_str)

In [ ]:
# print('dU_dw')
# expr = dUdw_S_simp.subs(ecc, 0)
# expr = expand(sync_spin(expr, host=True, sat=True) / (sign(mean_n)))
# for mode, love_S in love_table_sat.items():
#     expr = collect(expr, love_S)
# disp(expr)
# latex_str = latex(expr)
# latex_str = latex_str.replace('\\Xi^{}_{S2}', '\\operatorname{K}_{j}')
# print(latex_str)

In [ ]:
# print('dU_dO')
# expr = dUdO_S_simp.subs(ecc, 0)
# expr = expand(sync_spin(expr, host=True, sat=True) / (sign(mean_n)))
# for mode, love_S in love_table_sat.items():
#     expr = collect(expr, love_S)
# disp(expr)
# latex_str = latex(expr)
# latex_str = latex_str.replace('\\Xi^{}_{S2}', '\\operatorname{K}_{j}')
# print(latex_str)

## General - Inclin Truncated

In [ ]:
# Inclin_trunc = 5

In [ ]:
# print('Full Potential')
# expr = full_potential_simp
# expr = taylor(expr, inclin_sat, Inclin_trunc)
# expr = expand(sync_spin(expr, host=True, sat=True))
# for mode, love_S in love_table_sat.items():
#     expr = collect(expr, love_S)
# disp(expr)
# latex_str = latex(expr)
# latex_str = latex_str.replace('\\Xi^{}_{S2}', '\\operatorname{K}_{j}')
# print(latex_str)

In [25]:
# print('Tidal Heating')
# expr = heating_S_simp
# expr = taylor(expr, ecc, 12)
# expr = expand(sync_spin(expr, host=True, sat=True) / abs(mean_n))
# # expr = taylor(expr, inclin_sat, Inclin_trunc)
# for mode, love_S in love_table_sat.items():
#     expr = collect(expr, love_S)
# disp(expr)
# latex_str = latex(expr)
# latex_str = latex_str.replace('\\Xi^{}_{S2}', '\\operatorname{K}_{j}')
# print(latex_str)

Tidal Heating


KeyboardInterrupt: 

In [ ]:
# print('dU_dM')
# expr = expand(sync_spin(expr, host=True, sat=True) / (sign(mean_n)))
# expr = taylor(expr, inclin_sat, Inclin_trunc)
# for mode, love_S in love_table_sat.items():
#     expr = collect(expr, love_S)
# disp(expr)
# latex_str = latex(expr)
# latex_str = latex_str.replace('\\Xi^{}_{S2}', '\\operatorname{K}_{j}')
# print(latex_str)

In [ ]:
# print('dU_dw')
# expr = expand(sync_spin(expr, host=True, sat=True) / (sign(mean_n)))
# expr = taylor(expr, inclin_sat, Inclin_trunc)
# for mode, love_S in love_table_sat.items():
#     expr = collect(expr, love_S)
# disp(expr)
# latex_str = latex(expr)
# latex_str = latex_str.replace('\\Xi^{}_{S2}', '\\operatorname{K}_{j}')
# print(latex_str)

In [ ]:
# print('dU_dO')
# expr = expand(sync_spin(expr, host=True, sat=True) / (sign(mean_n)))
# expr = taylor(expr, inclin_sat, Inclin_trunc)
# for mode, love_S in love_table_sat.items():
#     expr = collect(expr, love_S)
# disp(expr)
# latex_str = latex(expr)
# latex_str = latex_str.replace('\\Xi^{}_{S2}', '\\operatorname{K}_{j}')
# print(latex_str)

In [24]:
# Clean up
heating_H = clean_up(heating_H)
torque_H = clean_up(torque_H)
dUdM_H = clean_up(dUdM_H)
dUdw_H = clean_up(dUdw_H)
dUdO_H = clean_up(dUdO_H)

for mode, love_H in love_table_host.items():
    heating_H = collect(heating_H, love_H)
    dUdM_H = collect(dUdM_H, love_H)
    dUdw_H = collect(dUdw_H, love_H)
    dUdO_H = collect(dUdO_H, love_H)

heating_S = clean_up(heating_S)
torque_S = clean_up(torque_S)
dUdM_S = clean_up(dUdM_S)
dUdw_S = clean_up(dUdw_S)
dUdO_S = clean_up(dUdO_S)

for mode, love_S in love_table_sat.items():
    heating_S = collect(heating_S, love_S)
    dUdM_S = collect(dUdM_S, love_S)
    dUdw_S = collect(dUdw_S, love_S)
    dUdO_S = collect(dUdO_S, love_S)

KeyboardInterrupt: 

## Find Disturbing Potential Derivatives

In [ ]:
dRdM = (1 / beta) * (-mass_host * dUdM_S - mass_sat * dUdM_H)
dRdw_host = (1 / beta) * (-mass_sat * dUdw_H)
dRdw_sat = (1 / beta) * (-mass_host * dUdw_S)
dRdO_host = (1 / beta) * (-mass_sat * dUdO_H)
dRdO_sat = (1 / beta) * (-mass_host * dUdO_S)

In [ ]:
dRdM = expand(clean_up(dRdM.subs(mass_host + mass_sat, mean_n**2 * semi_major_axis**3 / newton_G)))
dRdw_host = expand(clean_up(dRdw_host.subs(mass_host + mass_sat, mean_n**2 * semi_major_axis**3 / newton_G)))
dRdw_sat = expand(clean_up(dRdw_sat.subs(mass_host + mass_sat, mean_n**2 * semi_major_axis**3 / newton_G)))
dRdO_host = expand(clean_up(dRdO_host.subs(mass_host + mass_sat, mean_n**2 * semi_major_axis**3 / newton_G)))
dRdO_sat = expand(clean_up(dRdO_sat.subs(mass_host + mass_sat, mean_n**2 * semi_major_axis**3 / newton_G)))

## Find Time Derivatives

In [ ]:
dadt = (2 / (mean_n * semi_major_axis)) * dRdM
dedt = ((1 - ecc**2) / (mean_n * semi_major_axis**2 * ecc)) * dRdM - \
    ((1 - ecc**2)**(Rational(1,2)) / (mean_n * semi_major_axis**2 * ecc)) * (dRdw_host + dRdw_sat)
dspin_dt_host = - (beta / moi_host) * dRdO_host
dspin_dt_sat = - (beta / moi_sat) * dRdO_sat

### I = 0, Spin Sync Case

In [ ]:
dadt_synci0 = sync_spin(dadt.subs(inclin_host, 0).subs(inclin_sat, 0), sat=True, host=True)
dedt_synci0 = sync_spin(dedt.subs(inclin_host, 0).subs(inclin_sat, 0), sat=True, host=True)
dspin_dt_host_synci0 = sync_spin(dspin_dt_host.subs(inclin_host, 0).subs(inclin_sat, 0), sat=True, host=True)
dspin_dt_sat_synci0 = sync_spin(dspin_dt_sat.subs(inclin_host, 0).subs(inclin_sat, 0), sat=True, host=True)

for mode, love_S in love_table_sat.items():
    dadt_synci0 = collect(dadt_synci0, love_S)
    dedt_synci0 = collect(dedt_synci0, love_S)
    dspin_dt_sat_synci0 = collect(dspin_dt_sat_synci0, love_S)
for mode, love_H in love_table_host.items():
    dadt_synci0 = collect(dadt_synci0, love_H)
    dedt_synci0 = collect(dedt_synci0, love_H)
    dspin_dt_host_synci0 = collect(dspin_dt_host_synci0, love_H)

In [ ]:
print('da/dt')
# disp(dadt_synci0)

latex_str = latex(dadt_synci0/(Rational(3,2)))
# latex_str = latex_str.replace('\\Xi^{}_{S2}', '\\operatorname{K}_{j}')
print(latex_str)

In [ ]:
print('de/dt')
disp(dedt_synci0)

In [ ]:
print('SpinDerivs')
disp(dspin_dt_sat_synci0)
disp(dspin_dt_host_synci0)